# Semantic Text Similarity

In [33]:
'''!pip install fasttext
import fasttext.util
fasttext.util.download_model('ca', if_exists='ignore')  # English
fasttext.load_model('cc.ca.100.bin')'''

"!pip install fasttext\nimport fasttext.util\nfasttext.util.download_model('ca', if_exists='ignore')  # English\nfasttext.load_model('cc.ca.100.bin')"

In [34]:
'''ft = fasttext.load_model('cc.ca.300.bin')
fasttext.util.reduce_model(ft, 100)
ft.save_model('cc.ca.100.bin.gz')'''

"ft = fasttext.load_model('cc.ca.300.bin')\nfasttext.util.reduce_model(ft, 100)\nft.save_model('cc.ca.100.bin.gz')"

In [35]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List

In [36]:
# Load stopwords in spanish
import nltk
from nltk.corpus import stopwords
#STOPWORDS_ES = {"yo", "tú", "él", "ella", "nosotros", "vosotros", "ellos", "de", "a", }
STOPWORDS_CAT = {"jo", "tu", "ell", "ella", "nosaltres", "vosaltres", "ells", "elles", "de", "a", }
#STOPWORDS_CAT = stopwords.words("catalan")

In [37]:
# Define preprocessing
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    preprocessed = [token for token in preprocessed if token not in STOPWORDS_CAT]
    return preprocessed

# Load Vectors

In [3]:
import os

path_actual = os.getcwd()
os.chdir("..")
carpeta_anterior = os.getcwd()
print("Nueva ruta:", carpeta_anterior)


Ruta actual: /Users/gibertantentas/Library/CloudStorage/OneDrive-UniversitatPolitècnicadeCatalunya/2n/Q2/PLH/Practica4/PLH-4
Nueva ruta: /Users/gibertantentas/Library/CloudStorage/OneDrive-UniversitatPolitècnicadeCatalunya/2n/Q2/PLH/Practica4


In [4]:
# WORD 2 VEC

WORD_EMBEDDING_FILE = os.path.join(carpeta_anterior, "cc.ca.300.bin.gz")
# Load with gensim
from gensim.models import fasttext
wv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)

#WORD_EMBEDDING_FILE = load_dataset("projecte-aina/catalan_general_crawling")

In [39]:
'''# ROBERTA
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
model = AutoModelForMaskedLM.from_pretrained("projecte-aina/roberta-base-ca-v2")'''

'# ROBERTA\nfrom transformers import AutoTokenizer, AutoModelForMaskedLM\n\ntokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")\nmodel = AutoModelForMaskedLM.from_pretrained("projecte-aina/roberta-base-ca-v2")'

In [40]:
'''# SPACY
!python -m spacy download ca_core_news_md
import spacy
model = spacy.load("ca_core_news_md")
wv_model = model.vocab.vectors
'''

'# SPACY\n!python -m spacy download ca_core_news_md\nimport spacy\nmodel = spacy.load("ca_core_news_md")\nwv_model = model.vocab.vectors\n'

In [41]:
# ONE HOT ENCODING

In [42]:
# If you want, you can use mmaps
USE_MMAP = False
if USE_MMAP:
    from gensim.models.fasttext import FastTextKeyedVectors
    MMAP_PATH = 'cc.ca.gensim.bin'
    # wv_model.save(MMAP_PATH)
    wv_model = FastTextKeyedVectors.load(MMAP_PATH, mmap='r')

In [43]:
# Sample data
input_pairs_example = [
    ('Me gusta el futbol', 'Disfruto viendo partidos de futbol', 4),
    ('El cielo está despejado', 'Hace un día bonito', 4.5),
    ('Me encanta viajar', 'Explorar nuevos lugares es una pasión', 3.5),
    ('Prefiero el verano', 'No me gusta el frío del invierno', 2.5),
    ('Tengo hambre', '¿Qué hay para cenar?', 2),
    ('La música me relaja', 'Escuchar música es una terapia', 3),
    ('El libro es emocionante', 'No puedo dejar de leerlo', 4),
    ('Me gusta la pizza', 'Es mi comida favorita', 4.5),
    ('Estoy cansado', 'Necesito hacer una siesta', 1.5),
    ('Hoy hace mucho calor', 'Es un día sofocante', 3.5)
]

In [44]:
# Real data
!pip install datasets
from datasets import load_dataset
#dataset = load_dataset("PlanTL-GOB-ES/sts-es")
dataset = load_dataset("projecte-aina/sts-ca")

/Users/gibertantentas/.pyenv/versions/3.11.5/envs/IAA/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [45]:
print(len(dataset['train']))

2073


In [46]:
dataset['train'] = dataset['train'].select(range(len(dataset['train']) // 200))
dataset['validation'] = dataset['validation'].select(range(len(dataset['validation']) // 200))
dataset['test'] = dataset['test'].select(range(len(dataset['test']) // 200))



In [47]:
print(len(dataset['train']))

10


In [48]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["test"].to_list()]

In [49]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [50]:
print(sentence_pairs[0])

(['atorga', 'per', 'primer', 'cop', 'les', 'mencions', 'encarna', 'sanahuja', 'la', 'inclusió', 'de', 'la', 'perspectiva', 'de', 'gènere', 'en', 'docència', 'universitària'], ['creen', 'la', 'menció', 'encarna', 'sanahuja', 'la', 'inclusió', 'de', 'la', 'perspectiva', 'de', 'gènere', 'en', 'docència', 'universitària'])


In [51]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [52]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights
    
def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        #print('ABANS', sentence_1)
        #print('ABANS', sentence_2)
        sentence_1_preproc = preprocess(sentence_1)
        sentence_2_preproc = preprocess(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            #print('DESPRES',sentence_1_preproc)
            #print('DESPRES',sentence_2_preproc)
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [53]:
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped_train = map_pairs(input_pairs,  tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_val = map_pairs(input_pairs_val, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_test = map_pairs(input_pairs_test, tf_idf_model=modelo_tfidf, dictionary=diccionario, )

In [54]:
for vectors, similitud in mapped[:5]:
    print(f"Pares de vectores: {vectors[0].shape}, {vectors[1].shape}")
    print(f"Puntuación de similitud: {similitud}")

Pares de vectores: (100,), (100,)
Puntuación de similitud: 3.5
Pares de vectores: (100,), (100,)
Puntuación de similitud: 1.25
Pares de vectores: (100,), (100,)
Puntuación de similitud: 3.6700000762939453
Pares de vectores: (100,), (100,)
Puntuación de similitud: 2.25
Pares de vectores: (100,), (100,)
Puntuación de similitud: 2.0


In [55]:
# Define the Model
import tensorflow as tf

def build_and_compile_model(embedding_size: int = 100, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 = first_projection(input_1)
    projected_2 = first_projection(input_2)
    
    # Compute the cosine distance using a Lambda layer
    def cosine_distance(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))

    output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adamax(learning_rate))
    return model

In [56]:
def build_and_compile_model_better(embedding_size: int = 300, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 =  first_projection(input_1)
    projected_2 = first_projection(input_2)
    
    # Compute the cosine distance using a Lambda layer
    def normalized_product(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return x1_normalized * x2_normalized

    output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
    output = tf.keras.layers.Dropout(0.1)(output)
    output = tf.keras.layers.Dense(
        16,
        activation="relu",
    )(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(
        1,
        activation="sigmoid",
    )(output)
    
    output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
    
    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))
    return model

In [57]:
# Define training constants
batch_size: int = 64
#num_epochs: int = 64
num_epochs: int = 2

In [58]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)

In [59]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [60]:
# Show shapes
x_train[0].shape, x_train[1].shape, y_train.shape

((10, 100), (10, 100), (10,))

In [61]:
# Construir y compilar el modelo
model = build_and_compile_model()
#tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
print(model.summary())

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 100)       │     10,100 │ input_layer_2[0]… │
│                     │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None)            │          0 │ dense_1[0][0],    │
│                     │                   │            │ dense_1[1][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,100 (39.45 KB)

 Trainable params: 10,100 (39.45 KB)

 Non-trainable params: 0 (0.00 B)

None


In [62]:
'''import fasttext

# Función para escribir el corpus en un archivo de texto
def write_corpus_to_file(input_pairs, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for sentence_1, sentence_2, _ in input_pairs:
            sentence_1_preproc = preprocess(sentence_1)
            sentence_2_preproc = preprocess(sentence_2)
            f.write(' '.join(sentence_1_preproc) + '\n')
            f.write(' '.join(sentence_2_preproc) + '\n')

# Escribir los corpus en archivos de texto
write_corpus_to_file(input_pairs, 'corpus_train.txt')
write_corpus_to_file(input_pairs_val, 'corpus_val.txt')
write_corpus_to_file(input_pairs_test, 'corpus_test.txt')

# Entrenar el modelo FastText utilizando el corpus de entrenamiento
model = fasttext.train_unsupervised(input='corpus_train.txt', model='skipgram', epoch=10, lr=0.1, dim=100, minCount=1, wordNgrams=1)

# Guardar el modelo entrenado
model.save_model('fasttext_model.bin')
'''

"import fasttext\n\n# Función para escribir el corpus en un archivo de texto\ndef write_corpus_to_file(input_pairs, file_path):\n    with open(file_path, 'w', encoding='utf-8') as f:\n        for sentence_1, sentence_2, _ in input_pairs:\n            sentence_1_preproc = preprocess(sentence_1)\n            sentence_2_preproc = preprocess(sentence_2)\n            f.write(' '.join(sentence_1_preproc) + '\n')\n            f.write(' '.join(sentence_2_preproc) + '\n')\n\n# Escribir los corpus en archivos de texto\nwrite_corpus_to_file(input_pairs, 'corpus_train.txt')\nwrite_corpus_to_file(input_pairs_val, 'corpus_val.txt')\nwrite_corpus_to_file(input_pairs_test, 'corpus_test.txt')\n\n# Entrenar el modelo FastText utilizando el corpus de entrenamiento\nmodel = fasttext.train_unsupervised(input='corpus_train.txt', model='skipgram', epoch=10, lr=0.1, dim=100, minCount=1, wordNgrams=1)\n\n# Guardar el modelo entrenado\nmodel.save_model('fasttext_model.bin')\n"

In [63]:
# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/2


In [ ]:
from scipy.stats import pearsonr
x_test, y_test = pair_list_to_x_y(mapped_test)

In [ ]:
# Baseline
def compute_pearson_baseline(x_, y_):
    y_pred_baseline = []
    for v1, v2 in zip(*x_):
        d = 1.0 - spatial.distance.cosine(v1, v2)
        y_pred_baseline.append(d)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred_baseline, y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (baseline-train): {compute_pearson_baseline(x_train, y_train)}")
print(f"Correlación de Pearson (baseline-validation): {compute_pearson_baseline(x_val, y_val)}")
print(f"Correlación de Pearson (baseline-test): {compute_pearson_baseline(x_test, y_test)}")

Correlación de Pearson (baseline-train): 0.6183244864535127
Correlación de Pearson (baseline-validation): 0.7813210222946088
Correlación de Pearson (baseline-test): 0.6335381615219905


In [ ]:
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = model.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}")

42/42 [==============================] - 0s 768us/step
Correlación de Pearson (train): 0.9304827865557197
3/3 [==============================] - 0s 1ms/step
Correlación de Pearson (validation): 0.6014932513445799
5/5 [==============================] - 0s 796us/step
Correlación de Pearson (test): 0.6084397809211268
